In [48]:
from obiwow.data_reader_parser import standardise_time_of_day_column, write_html_page
from obiwow.data_reader_parser import (
    parse_yaml, parse_csv_to_pandas, merge_submission_schedule,
    add_start_end_time_to_schedule, annotate_networking_event, write_ical_files, write_schedule_json
)
from obiwow.tsv_to_html import generate_workshop_body, generate_schedule_table, generate_full_html_page

In [49]:
def import_all_config() -> dict:
    """
    Import all configuration files.

    Returns:
        dict: A dictionary containing all configuration data.
    """
    paths_config = parse_yaml('config/paths.yaml')
    yearly_config = parse_yaml('config/yearly_config.yaml')
    nettskjema_columns = parse_yaml('config/nettskjema_columns.yaml')
    schedule_columns = parse_yaml('config/schedule_columns.yaml')
    rooms_config = parse_yaml('config/rooms.yaml')

    return {
        'paths': paths_config,
        'yearly': yearly_config,
        'nettskjema_columns': nettskjema_columns,
        'schedule_columns': schedule_columns,
        'rooms': rooms_config
    }

In [50]:
config = import_all_config()

In [51]:
paths = config['paths']

In [52]:
config = import_all_config()

registration_open = config['yearly'].get('registration_open', False)
paths = config['paths']
yearly = config['yearly']
nettskjema_columns = config['nettskjema_columns']
schedule_columns = config['schedule_columns']
rooms = config['rooms']

df_submissions = parse_csv_to_pandas(paths['input']['survey_results']['file_path'],
                                         paths['input']['survey_results']['delimiter'])
df_schedule = parse_csv_to_pandas(paths['input']['schedule']['file_path'],
                                      paths['input']['schedule']['delimiter'])

# Remove rows with cancelled workshops
df_schedule = df_schedule[df_schedule[schedule_columns['status_column']] != 'cancelled']

standardise_time_of_day_column(df_schedule, schedule_columns)
df_schedule = add_start_end_time_to_schedule(df_schedule, schedule_columns)
df_schedule = annotate_networking_event(df_schedule, schedule_columns)

df_merge_submission_schedule = merge_submission_schedule(df_submissions, df_schedule, nettskjema_columns,
                                                         schedule_columns)

In [53]:
import unicodedata
import re
import os
import pandas as pd

def slugify(value, allow_unicode=False):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')


In [104]:
for index, row in df_merge_submission_schedule.iterrows():
    title = row['Title of the workshop']
    my_date = datetime.datetime.strptime(row["Date"], "%d.%m.%y")
    weekday = str(my_date.strftime('%d'))
    day_of_week = calendar.day_name[my_date.weekday()]  #'Wednesday'
    description = row['A short description of your workshop (max 250 words)']
    if not pd.isna(row['Instructor 2']):
        instructors = ", ".join([row['Instructor 1'], row['Instructor 2']])
    else:
        instructors = row['Instructor 1']
    if not pd.isna(row['Title of the workshop']):
        slug = slugify(title)
        path = "/".join(["../OBiWoW-2024", "-".join([weekday, day_of_week]), slug])
        if not os.path.exists(path): 
            os.makedirs(path)
        text = "\n".join(["# %s" % title, 
                         description, 
                         "## Instructors",
                         instructors,
                         "## Live Troubleshooting Session", 
                         "## Software Requirements"])
        with open("/".join([path, "README.md"]), "w") as text_file:
            text_file.write(text)